<a href="https://colab.research.google.com/github/dungdt-infopstats/TV-command-synthesis/blob/main/notebooks/Speaker_DataSplit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Prep

In [ ]:
# aug speech
!gdown 1s0iFSfi-vnadzKuYz6mHIpXrdJw9YDWa

Downloading...
From (original): https://drive.google.com/uc?id=1s0iFSfi-vnadzKuYz6mHIpXrdJw9YDWa
From (redirected): https://drive.google.com/uc?id=1s0iFSfi-vnadzKuYz6mHIpXrdJw9YDWa&confirm=t&uuid=8d250119-86d0-4f1a-b3f9-8103024bb752
To: /content/aug_speech.zip
100% 407M/407M [00:09<00:00, 43.1MB/s]


In [ ]:
!unzip -q aug_speech.zip

In [ ]:
import json

list_file = []
for folder in os.listdir('concat_speech/augmented'):
    for file in os.listdir(os.path.join('concat_speech/augmented', folder)):
        if file.endswith('.json'):
            with open(os.path.join('concat_speech/augmented', folder, file), 'r') as f:
                data = json.load(f)
                file_id = folder
                list_file.append(
                    {
                        'file_id': file_id,
                    }
                )


In [ ]:
df_file = pd.DataFrame(list_file)

# Get Speaker

In [ ]:
!gdown 10uHbeqAO5uqOQV6eOXnTuS150MEUTDdk

Downloading...
From (original): https://drive.google.com/uc?id=10uHbeqAO5uqOQV6eOXnTuS150MEUTDdk
From (redirected): https://drive.google.com/uc?id=10uHbeqAO5uqOQV6eOXnTuS150MEUTDdk&confirm=t&uuid=ea3edfc4-ee71-40e6-af2a-4827dbf20d74
To: /content/synthesis.zip
100% 955M/955M [00:14<00:00, 68.0MB/s]


In [ ]:
!unzip -q synthesis.zip

In [ ]:
import os

for file in os.listdir('synthesis'):
    !unzip -q {os.path.join('synthesis', file)} -d {os.path.join('synthesis_all')}

In [ ]:
import json
import pandas as pd
speaker_list = []

for folder in os.listdir('synthesis_all'):
    for file in os.listdir(os.path.join('synthesis_all', folder)):
        if file.endswith('.json'):
            with open(os.path.join('synthesis_all', folder, file), 'r') as f:
                data = json.load(f)
                speech_id = data['id']
                speaker_id = data['ref_file'].split('/')[1].split('_')[0]
                speaker_file = data['ref_file'].split('/')[1]
                type_text = data['type']
                speaker_list.append(
                    {
                        'speech_id': speech_id,
                        'speaker_id': speaker_id,
                        'speaker_file': speaker_file,
                        'type': type_text
                    }
                )

In [ ]:
df_speaker = pd.DataFrame(speaker_list)

In [ ]:
df_speaker

,speech_id,speaker_id,speaker_file,type
0,920d3ac0,p286,p286_446.wav,chain_mix
1,d4b0bd59,p233,p233_049.wav,chain_active
2,3b468809,p310,p310_335.wav,single_mix
3,447842a6,p298,p298_332.wav,single_mix
4,05ec6483,p341,p341_361.wav,chain_active
...,...,...,...,...
3495,f685e16f,p276,p276_378.wav,chain_active
3496,5785fd32,p254,p254_390.wav,non_active
3497,ef941329,p339,p339_146.wav,single_mix
3498,e2bfc1ef,p364,p364_072.wav,single_mix


In [ ]:
df_speaker.to_csv('speaker_statistic_vctk.csv', index=False)

In [ ]:
df_speaker['file_id'] = df_speaker['type'] + '_' + df_speaker['speech_id']

# Merge

In [ ]:
df_merge = df_file.merge(df_speaker, on='file_id', how='left')

In [ ]:
df_merge

,file_id,speech_id,speaker_id,speaker_file,type
0,chain_active_d4b0bd59,d4b0bd59,p233,p233_049.wav,chain_active
1,single_mix_3b468809,3b468809,p310,p310_335.wav,single_mix
2,chain_active_05ec6483,05ec6483,p341,p341_361.wav,chain_active
3,non_active_87c544b3,87c544b3,p339,p339_004.wav,non_active
4,chain_mix_391a13d6,391a13d6,p228,p228_132.wav,chain_mix
...,...,...,...,...,...
3016,chain_mix_3010fc57,3010fc57,p248,p248_085.wav,chain_mix
3017,single_active_4125f7f7,4125f7f7,p304,p304_420.wav,single_active
3018,chain_active_f685e16f,f685e16f,p276,p276_378.wav,chain_active
3019,non_active_5785fd32,5785fd32,p254,p254_390.wav,non_active


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import jensenshannon  # dùng JS-divergence cho dễ

def best_speaker_split(df, test_size=0.2, n_trials=100, random_state=42):
    speaker_ids = df["speaker_id"].unique()
    best_score = float("inf")
    best_split = None

    for seed in range(random_state, random_state + n_trials):
        train_speakers, test_speakers = train_test_split(
            speaker_ids, test_size=test_size, random_state=seed
        )
        train_df = df[df["speaker_id"].isin(train_speakers)]
        test_df  = df[df["speaker_id"].isin(test_speakers)]

        # phân bố type
        p_train = train_df["type"].value_counts(normalize=True).sort_index()
        p_test  = test_df["type"].value_counts(normalize=True).sort_index()

        # align index
        all_types = sorted(set(df["type"]))
        p_train = p_train.reindex(all_types, fill_value=0)
        p_test  = p_test.reindex(all_types, fill_value=0)

        # JS divergence (càng nhỏ càng giống nhau)
        score = jensenshannon(p_train, p_test)

        if score < best_score:
            best_score = score
            best_split = (train_speakers, test_speakers, p_train, p_test)

    return best_split, best_score

# --- dùng thử ---
best_split, score = best_speaker_split(df_merge, test_size=0.2, n_trials=200)

train_speakers, test_speakers, p_train, p_test = best_split

train_df = df_merge[df_merge["speaker_id"].isin(train_speakers)].reset_index(drop=True)
test_df  = df_merge[df_merge["speaker_id"].isin(test_speakers)].reset_index(drop=True)

print("Best JS divergence:", score)
print("Train distribution:\n", p_train)
print("Test distribution:\n", p_test)

Best JS divergence: 0.0035702499214655295
Train distribution:
 type
chain_active     0.224380
chain_mix        0.193388
non_active       0.231405
single_active    0.178099
single_mix       0.172727
Name: proportion, dtype: float64
Test distribution:
 type
chain_active     0.221298
chain_mix        0.194676
non_active       0.229617
single_active    0.179700
single_mix       0.174709
Name: proportion, dtype: float64


In [ ]:
train_df.to_csv('train_df.csv', index=False)
test_df.to_csv('test_df.csv', index=False)

In [ ]:
set_train = set(train_df['speaker_id'].unique())

In [ ]:
set_test = set(test_df['speaker_id'].unique())

In [ ]:
# overlap checking

set_train & set_test

set()

In [ ]:
import os
import shutil

def split_folders(base_dir, output_dir, train_df, test_df):
    os.makedirs(os.path.join(output_dir, "train", "aug"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "test", "aug"), exist_ok=True)

    # train
    for fid in train_df["file_id"]:
        src = os.path.join(base_dir, fid)
        dst = os.path.join(output_dir, "train", "aug", fid)
        if os.path.exists(src):
            shutil.copytree(src, dst)

    # test
    for fid in test_df["file_id"]:
        src = os.path.join(base_dir, fid)
        dst = os.path.join(output_dir, "test", "aug", fid)
        if os.path.exists(src):
            shutil.copytree(src, dst)

# --- dùng thử ---
base_dir = "concat_speech/augmented"           # thư mục gốc chứa file_id
output_dir = "dataset_split"   # nơi lưu train/test

split_folders(base_dir, output_dir, train_df, test_df)


In [ ]:
!zip vctk_split.zip dataset_split -r

Streaming output truncated to the last 5000 lines.
  adding: dataset_split/train/aug/single_active_0fb18099/ (stored 0%)
  adding: dataset_split/train/aug/single_active_0fb18099/single_active_0fb18099_aug.wav (deflated 20%)
  adding: dataset_split/train/aug/single_active_0fb18099/single_active_0fb18099_aug.json (deflated 48%)
  adding: dataset_split/train/aug/chain_mix_3840cef2/ (stored 0%)
  adding: dataset_split/train/aug/chain_mix_3840cef2/chain_mix_3840cef2_aug.json (deflated 54%)
  adding: dataset_split/train/aug/chain_mix_3840cef2/chain_mix_3840cef2_aug.wav (deflated 16%)
  adding: dataset_split/train/aug/chain_active_53756315/ (stored 0%)
  adding: dataset_split/train/aug/chain_active_53756315/chain_active_53756315_aug.json (deflated 48%)
  adding: dataset_split/train/aug/chain_active_53756315/chain_active_53756315_aug.wav (deflated 19%)
  adding: dataset_split/train/aug/single_active_6f14786d/ (stored 0%)
  adding: dataset_split/train/aug/single_active_6f14786d/single_active_6f